# pytorch_mnist.ipynb
# WESmith 06/26/23
## review of MNIST processing with pytorch
## see https://nextjournal.com/gkoehler/pytorch-mnist

In [ ]:
import torch
import torch.nn            as nn
import torch.nn.functional as F
import torch.optim         as optim
import torchvision
import numpy               as np
import matplotlib.pyplot   as plt

In [ ]:
n_epochs         = 3
batch_size_train = 64
batch_size_test  = 1000
learning_rate    = 0.01
momentum         = 0.5
log_interval     = 10  # WS (was 10)

random_seed      = 42
torch.backends.cudnn.enabled = False  # disable nondeterministic algorithms
torch.manual_seed(random_seed)

In [ ]:
#torchvision.datasets.MNIST?

In [ ]:
#torch.utils.data.DataLoader?

In [ ]:
data_dir = 'data'

In [ ]:
train_data = torchvision.datasets.MNIST(data_dir, train=True, download=True,
                                         transform=torchvision.transforms.Compose([
                                             torchvision.transforms.ToTensor(),
                                             torchvision.transforms.Normalize(
                                             (0.1307,), (0.3081))]))  # predefined global mean, std

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size_train, shuffle=True)

In [ ]:
test_data  = torchvision.datasets.MNIST(data_dir, train=False, download=True,
                                         transform=torchvision.transforms.Compose([
                                             torchvision.transforms.ToTensor(),
                                             torchvision.transforms.Normalize(
                                             (0.1307,), (0.3081))]))  # predefined global mean, std

In [ ]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size_test, shuffle=True)

In [ ]:
examples = enumerate(test_loader)  # create an emumerate object: it yields a tuple of (count, object)

In [ ]:
batch_idx, (example_data, example_targets) = next(examples)

In [ ]:
example_data.shape, example_targets.shape

In [ ]:
fig = plt.figure(figsize=(15, 12))
nr = 5
nc = 6
for i in range(nr * nc):
    plt.subplot(nr, nc, i+1)
    plt.tight_layout()
    plt.imshow(example_data[i][0], cmap='gray', interpolation=None)
    plt.title(f'idx: {i}, truth: {example_targets[i]}', c='w')
    plt.xticks([])
    plt.yticks([])

In [ ]:
example_data.mean(), example_data.std()  # almost at 0.0, 1.0

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d( 1, 10, kernel_size=5) #  1 channel coming in,  10 channels out
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # 10 channels coming in, 20 channels out
        self.conv2_drop = nn.Dropout2d()
        self.fc1   = nn.Linear(320, 50) # fully connected 4 x 4 x 20 = 320 inputs, 50 outputs
        self.fc2   = nn.Linear( 50, 10) # fully connected, 50 inputs, 10 outputs, 1 for each integer
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)  # self.training: Boolean, sets training or eval mode
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)  # WS added dim=1 to avoid throwing warnings

In [ ]:
network   = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

In [ ]:
train_losses  = []
train_counter = []
test_losses   = []
test_counter  = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [ ]:
test_counter

In [ ]:
def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss   = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            ll = len(train_loader.dataset)
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * len(data), ll,
                  100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append((batch_idx * 64) + ((epoch - 1) * ll))
            torch.save(network.state_dict(),   'results/model.pth')
            torch.save(optimizer.state_dict(), 'results/optimizer.pth')

In [ ]:
def test():
    network.eval()
    test_loss = 0
    correct   = 0
    with torch.no_grad():
        for data, target in test_loader:
            output     = network(data)
            # nll: negative log likelihood, size_average is deprecated, 'reduction' default is 'mean'
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred       = output.data.max(1, keepdim=True)[1]
            correct   += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
          test_loss, correct, len(test_loader.dataset),
          100. * correct / len(test_loader.dataset)))

In [ ]:
test() # before training, about 8% accuracy

In [ ]:
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()

In [ ]:
fig = plt.figure(figsize=(14, 6))
plt.plot(train_counter, train_losses, color='blue')
plt.scatter(test_counter, test_losses, color='red')
plt.grid()
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')

In [ ]:
with torch.no_grad():
    output = network(example_data)

In [ ]:
fig = plt.figure()
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.tight_layout()
    plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
    plt.title("Prediction: {}".format(
    output.data.max(1, keepdim=True)[1][i].item()), c='w')
    plt.xticks([])
    plt.yticks([])

In [ ]:
# demo of how to keep training from checkpoints:
network_state_dict = torch.load('results/model.pth')
network = Net()
network.load_state_dict(network_state_dict)

optimizer_state_dict = torch.load('results/optimizer.pth')
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
optimizer.load_state_dict(optimizer_state_dict)

In [ ]:
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()

In [ ]:
with torch.no_grad():
    output = network(example_data)

In [ ]:
len(output)

In [ ]:
for k in range(5):
    print(output[k].argmax().item(), example_targets[k].item())

In [ ]:
# WS create a precision matrix showing confusion errors
def precision(loader):  # loader is train_loader or test_loader
    dd = torch.zeros(10, 10) # row is truth, col is prediction
    network.eval()
    with torch.no_grad():
        for data, target in loader:
            output = network(data)
            for k in range(len(output)):
                pred  = output[k].argmax().item()
                truth = target[k].item()
                dd[truth, pred] += 1
    count = 0
    for k in range(10):
        count += dd[k, k]
    print(f'accuracy = {100. * count / dd.sum():.4f}')
    return dd.numpy()

In [ ]:
test_matrix = precision(test_loader)

In [ ]:
train_matrix = precision(train_loader)

In [ ]:
plt.imshow(np.log(test_matrix + 1))

In [ ]:
plt.imshow(np.log(train_matrix + 1))

In [ ]:
for k0, k1 in test_loader:
    print(k0.shape, k1.shape)